# Steam Data Cleaning

In [1]:
# standard library imports
from ast import literal_eval
import itertools
import time
import re

# third-party imports
import numpy as np
import pandas as pd

# customisations
pd.options.display.max_columns = 100

In [2]:
# read in downloaded data
raw_steam_data = pd.read_csv('../data/download/games.csv')

# print out number of rows and columns
print('Rows:', raw_steam_data.shape[0])
print('Columns:', raw_steam_data.shape[1])

# view first five rows
raw_steam_data.head()

Rows: 85103
Columns: 39


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://www.galacticbowling.net,NaN,NaN,True,False,False,0,NaN,0,6,11,NaN,30,0,NaN,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://trainbandit.com,NaN,support@rustymoyher.com,True,True,False,0,NaN,0,53,5,NaN,12,0,NaN,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,ramoncampiaof31@gmail.com,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://henosisgame.com/,https://henosisgame.com/,info@henosisgame.com,True,True,True,0,NaN,0,3,0,NaN,0,0,NaN,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,https://www.unusual-games.com/home/,https://www.unusual-games.com/contact/,welistentoyou@unusual-games.com,True,True,False,0,NaN,0,50,8,NaN,17,0,This Game may contain content not appropriate ...,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [3]:
null_counts = raw_steam_data.isnull().sum()
null_counts

AppID                             0
Name                              6
Release date                      0
Estimated owners                  0
Peak CCU                          0
Required age                      0
Price                             0
DLC count                         0
About the game                 3567
Supported languages               0
Full audio languages              0
Reviews                       75360
Header image                      0
Website                       45651
Support url                   43517
Support email                 13596
Windows                           0
Mac                               0
Linux                             0
Metacritic score                  0
Metacritic url                81191
User score                        0
Positive                          0
Negative                          0
Score rank                    85059
Achievements                      0
Recommendations                   0
Notes                       

In [4]:
threshold = raw_steam_data.shape[0] // 2

print('Drop columns with more than {} missing rows'.format(threshold))
print()

drop_rows = list(raw_steam_data.columns[null_counts > threshold])

drop_rows += ['Full audio languages', 'Metacritic score', 'Support email']

print('Columns to drop: {}'.format(drop_rows))

Drop columns with more than 42551 missing rows

Columns to drop: ['Reviews', 'Website', 'Support url', 'Metacritic url', 'Score rank', 'Notes', 'Full audio languages', 'Metacritic score', 'Support email']


In [5]:
raw_steam_data[(raw_steam_data['Name'].isnull()) | (raw_steam_data['Name'] == 'none')]

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Full audio languages,Reviews,Header image,Website,Support url,Support email,Windows,Mac,Linux,Metacritic score,Metacritic url,User score,Positive,Negative,Score rank,Achievements,Recommendations,Notes,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
2701,385020,none,"Nov 4, 2015",20000 - 50000,1,0,0.00,0,- discontinued - (please remove),"['English', 'French', 'Italian', 'German', 'Sp...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,True,True,0,NaN,0,13,32,NaN,0,0,NaN,276,0,288,0,none,NaN,"Single-player,Multi-player,Cross-Platform Mult...","Casual,Indie,Massively Multiplayer,Strategy","Indie,Strategy,Casual,Massively Multiplayer,Ch...",NaN,NaN
25222,396420,NaN,"Nov 1, 2016",0 - 0,0,0,0.00,0,Spookeningは3Dの恐怖ゲームで、あなたは毎夜に死んでゴーストとして復活します。 村...,[],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31613,339860,none,"Feb 27, 2015",0 - 0,0,0,0.00,0,NaN,['English'],['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,3,0,NaN,0,0,0,0,NaN,NaN,"Single-player,Multi-player,MMO,Steam Achieveme...","Adventure,Indie,Massively Multiplayer,RPG,Stra...",NaN,NaN,NaN
47174,398970,none,"Nov 5, 2015",50000 - 100000,0,0,0.00,0,NaN,['English'],['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,True,True,0,NaN,0,8,23,NaN,35,0,NaN,268,0,274,0,none,none,"Single-player,Multi-player,MMO,Online PvP,Stea...","Adventure,Indie,Massively Multiplayer,RPG,Stra...","Massively Multiplayer,Strategy,RPG,Indie,Adven...",NaN,NaN
72038,1116910,NaN,"Sep 25, 2019",0 - 0,0,0,6.99,0,NaN,[],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,0,0,The content contained in this game may not be ...,0,0,0,0,NaN,NaN,"Single-player,Steam Cloud","Action,Adventure,Casual,Indie,RPG,Simulation,S...",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
77215,1365520,NaN,"Aug 30, 2020",0 - 0,0,0,0.00,0,NaN,"['English', 'French', 'Italian', 'German', 'Sp...",[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,0,0,NaN,0,0,0,0,NaN,NaN,"Single-player,Steam Cloud",Early Access,NaN,NaN,NaN
77487,1080790,NaN,"Jul 12, 2019",0 - 0,0,0,0.00,0,NaN,['English'],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,20,0,NaN,0,0,0,0,NaN,NaN,NaN,"Adventure,Casual,Free to Play,Indie,RPG",NaN,NaN,NaN
77561,1256960,NaN,"Mar 11, 2020",0 - 20000,0,0,0.00,0,NaN,['English'],['English'],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,False,False,0,NaN,0,0,0,NaN,0,0,The game includes nudity and blood All charact...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77721,1172120,NaN,"Jan 23, 2020",0 - 0,0,0,0.00,0,NaN,['English'],[],NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,NaN,NaN,NaN,True,True,True,0,NaN,0,0,0,NaN,0,0,This game includes cartoon violence and blood.,0,0,0,0,NaN,NaN,Single-player,"Action,Indie,Simulation",NaN,NaN,NaN


In [6]:
def drop_null_cols(df, thresh=0.5):
    
    return df.drop(drop_rows, axis = 1)


def process_name_type(df):
    """Remove null values in name and type columns, and remove type column."""
    
    df = df[df['Name'].notnull()]
    df = df[df['Name'] != 'none']
    
    
    return df


def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process rest of columns
    df = process_name_type(df)
    
    return df

print(raw_steam_data.shape)
initial_processing = process(raw_steam_data)
print(initial_processing.shape)
initial_processing.head()

(85103, 39)
(85094, 30)


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Header image,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,6,11,30,0,0,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,53,5,12,0,0,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,0,0,0,0,0,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,True,0,3,0,0,0,0,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,50,8,17,0,0,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


## Processing Age

In [7]:
initial_processing['Required age'].value_counts(dropna=False).sort_index()

Required age
0     83454
1         1
3         8
5         1
6         6
7         7
9         1
10       35
11        1
12       34
13      204
14        6
15        8
16       68
17      919
18      333
19        1
20        2
21        5
Name: count, dtype: int64

In [8]:
def process_age(df):
    """Format ratings in age column to be in line with the PEGI Age Ratings system."""
    # PEGI Age ratings: 3, 7, 12, 16, 18
    cut_points = [-1, 0, 3, 7, 12, 16, 2000]
    label_values = [0, 3, 7, 12, 16, 18]
    
    df['Required age'] = pd.cut(df['Required age'], bins=cut_points, labels=label_values)
    
    return df


age_df = process_age(initial_processing)
age_df['Required age'].value_counts().sort_index()

Required age
0     83454
3         9
7        14
12       71
16      286
18     1260
Name: count, dtype: int64

## Processing Price

In [9]:
age_df['Price'].head()

0    19.99
1     0.99
2     4.99
3     5.99
4     0.00
Name: Price, dtype: float64

In [10]:
age_df['Price'].isnull().sum()

0

In [11]:
def print_steam_links(df):
    """Print links to store page for apps in a dataframe."""
    url_base = "https://store.steampowered.com/app/"
    
    for i, row in df.iterrows():
        appid = row['AppID']
        name = row['Name']
        
        print(name + ':', url_base + str(appid))
        

print_steam_links(age_df[:5])

Galactic Bowling: https://store.steampowered.com/app/20200
Train Bandit: https://store.steampowered.com/app/655370
Jolt Project: https://store.steampowered.com/app/1732930
Henosis™: https://store.steampowered.com/app/1355720
Two Weeks in Painland: https://store.steampowered.com/app/1139950


In [12]:
age_df[age_df['Name'].str.contains("BioShock™")]

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Header image,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
2963,409710,BioShock™ Remastered,"Sep 15, 2016",2000000 - 5000000,449,18,19.99,0,BioShock is a shooter unlike any you've ever p...,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,34221,8389,65,33902,360,11,135,21,"2K Boston,2K Australia,Blind Squirrel,Feral In...","2K,Feral Interactive (Mac)","Single-player,Steam Achievements,Full controll...","Action,RPG","FPS,Story Rich,Atmospheric,Horror,Singleplayer...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
27606,409720,BioShock™ 2 Remastered,"Sep 15, 2016",1000000 - 2000000,257,18,19.99,1,BioShock 2 provides players with the perfect b...,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,12850,5719,53,14672,397,0,160,0,"2K Marin,2K China,Digital Extremes,2K Australi...","2K,Feral Interactive (Mac)","Single-player,Steam Achievements,Full controll...","Action,RPG","FPS,Action,Singleplayer,Story Rich,Atmospheric...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN
32406,7670,BioShock™,"Aug 21, 2007",2000000 - 5000000,71,18,0.00,0,BioShock is a shooter unlike any you've ever p...,"['English', 'French', 'German', 'Italian', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,26231,1546,0,21456,345,0,109,0,"2K Boston,2K Australia",2K,"Single-player,Partial Controller Support","Action,RPG","FPS,Atmospheric,Story Rich,Action,Singleplayer...",https://cdn.akamai.steamstatic.com/steam/apps/...,NaN


In [13]:
age_df['Price'][0]

19.99

In [14]:
age_df['Price'][37]

7.99

## Processing Developers and Publishers

In [15]:
print('Developers null counts:', age_df['Developers'].isnull().sum())
print('Developers empty list counts:', age_df[age_df['Developers'] == "['']"].shape[0])

print('\nPublishers null counts:', age_df['Publishers'].isnull().sum())
print('Publishers empty list counts:', age_df[age_df['Publishers'] == "['']"].shape[0])

Developers null counts: 3580
Developers empty list counts: 0

Publishers null counts: 3859
Publishers empty list counts: 0


In [16]:
no_dev = age_df[age_df['Developers'].isnull()]

print('Total games missing developer:', no_dev.shape[0], '\n')

print_steam_links(no_dev[:5])

no_pub = age_df[age_df['Publishers'] == "['']"]

print('\nTotal games missing publisher:', no_pub.shape[0], '\n')
print_steam_links(no_pub[:5])

no_dev_or_pub = age_df[(age_df['Developers'].isnull()) & (age_df['Publishers'] == "['']")]

print('\nTotal games missing developer and publisher:', no_dev_or_pub.shape[0], '\n')
print_steam_links(no_dev_or_pub[:5])

Total games missing developer: 3580 

溪风谷之战 Playtest: https://store.steampowered.com/app/1943590
Burial Stone Playtest: https://store.steampowered.com/app/1966960
Emperial Knights Playtest: https://store.steampowered.com/app/1688630
Slotracers VR Playtest: https://store.steampowered.com/app/1478660
Pirates of the Asteroid Belt Playtest: https://store.steampowered.com/app/1613340

Total games missing publisher: 0 


Total games missing developer and publisher: 0 



In [17]:
age_df[['Developers', 'Publishers']].iloc[24:28]

,Developers,Publishers
24,SGデベロッパー,SGデベロッパー
25,PseudoGames,PseudoGames
26,Zero One,Zero One
27,Mens Sana Interactive,Mens Sana Interactive


In [18]:
def process_developers_and_publishers(df):
    # remove rows with missing data
    df = df[(df['Developers'].notnull()) & (df['Publishers'].notnull())].copy()
process_developers_and_publishers(age_df)

In [19]:
', '.join(['one item'])

'one item'

In [20]:
', '.join(['multiple', 'different', 'items'])

'multiple, different, items'

In [21]:
age_df.loc[age_df['Developers'].str.contains(",", na=False), ['AppID', 'Developers', 'Publishers']].head(4)

,AppID,Developers,Publishers
11,485000,"Wise Wizard Games, LLC","Wise Wizard Games, LLC"
35,1195610,"FAYEZ ALRASHEEDI,WADE AIZOUKY,KHALED ALRASHEEDI",THE MARCH OF KWF
41,866150,"Garage Games,Jan Hess",Garage Games
47,552520,"Ubisoft Montreal,Red Storm,Ubisoft Shanghai,Ub...",Ubisoft


In [22]:
age_df.loc[age_df['Developers'].str.contains(";", na=False), ['AppID', 'Developers', 'Publishers']]

,AppID,Developers,Publishers
6450,460210,bool games;,bool games;
20512,665890,Semicolon;,Semicolon;
41287,1271450,"Justin Buzon (Programmer; Artist),D'lorean Lat...",Sama Learning
47008,568480,";),Quickdraw Studios",Quickdraw Studios
54946,980770,poliko; entertainment,poliko; entertainment
64138,2119440,MasterGames Co;Ltd;,MasterGames Co;Ltd;
70004,1999920,7th;MINT,DoubleChallenge Inc.
84050,2686590,"Husky-Leaf Studios,Sku;Te",Husky-Leaf Studios
84257,2609920,Project;Hypatios Team,Project;Hypatios Team


In [23]:
age_df[(age_df['Publishers'] == "NA") | (age_df['Publishers'] == "N/A")].shape[0]

0

In [24]:
def process_developers_and_publishers(df):
    """Parse columns as semicolon-separated string."""
    # remove rows with missing data (~ means not)
    df = df[(df['Developers'].notnull()) & (df['Publishers'].notnull())].copy()
    df = df[~(df['Developers'].str.contains(';')) & ~(df['Publishers'].str.contains(';'))]
    
    # create list for each
    df['Developer'] = df['Developers']
    df['Publisher'] = df['Publishers']

    df = df.drop(['Developers', 'Publishers'], axis=1)
    
    return df

dev_pub_df = process_developers_and_publishers(age_df)
dev_pub_df[['Name', 'AppID', 'Developer', 'Publisher']].head()

,Name,AppID,Developer,Publisher
0,Galactic Bowling,20200,Perpetual FX Creative,Perpetual FX Creative
1,Train Bandit,655370,Rusty Moyher,Wild Rooster
2,Jolt Project,1732930,Campião Games,Campião Games
3,Henosis™,1355720,Odd Critter Games,Odd Critter Games
4,Two Weeks in Painland,1139950,Unusual Games,Unusual Games


## Processing Categories and Genres

In [25]:
print('Categories:\n')
print('Null values:', dev_pub_df['Categories'].isnull().sum())
print()
print(dev_pub_df['Categories'][0])

print('\nGenres:\n')
print('Null values:', dev_pub_df['Genres'].isnull().sum())
print()
print(dev_pub_df['Genres'].iloc[0])
print(dev_pub_df['Genres'].iloc[1000])

Categories:

Null values: 1164

Single-player,Multi-player,Steam Achievements,Partial Controller Support

Genres:

Null values: 78

Casual,Indie,Sports
Free to Play


In [26]:
print_steam_links(dev_pub_df[dev_pub_df['Categories'].isnull()].sample(5, random_state=0))

Moon VR Video Player: https://store.steampowered.com/app/705160
Tom Clancy's Ghost Recon® Island Thunder™: https://store.steampowered.com/app/13630
Starry Makino: https://store.steampowered.com/app/736700
Mannerheim Virtual Experience: https://store.steampowered.com/app/1179090
Vitamins: https://store.steampowered.com/app/2187210


In [27]:
print_steam_links(dev_pub_df[dev_pub_df['Genres'].isnull()].sample(5, random_state=0))

Fire and Steel: https://store.steampowered.com/app/1791790
M.I.A. - Overture: https://store.steampowered.com/app/712060
ChessBase 13 Pro: https://store.steampowered.com/app/377350
Magic Flight Academy: https://store.steampowered.com/app/867580
Hollywood Hills Mansion: https://store.steampowered.com/app/451450


In [28]:
example_category = "[{'id': 1, 'description': 'Multi-player'}, {'id': 36, 'description': 'Online Multi-Player'}, {'id': 37, 'description': 'Local Multi-Player'}]"

[x['description'] for x in literal_eval(example_category)]

['Multi-player', 'Online Multi-Player', 'Local Multi-Player']

In [29]:
def process_categories_and_genres(df):
    df = df.copy()
    df = df[(df['Categories'].notnull()) & (df['Genres'].notnull()) & (df['Tags'].notnull())]
    
    return df

cat_gen_df = process_categories_and_genres(dev_pub_df)
cat_gen_df[['AppID', 'Categories', 'Genres']].head()

,AppID,Categories,Genres
0,20200,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports"
1,655370,"Single-player,Steam Achievements,Full controll...","Action,Indie"
3,1355720,"Single-player,Full controller support","Adventure,Casual,Indie"
4,1139950,"Single-player,Steam Achievements","Adventure,Indie"
5,1469160,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip..."


## Processing Achievements and Content Descriptors

In [30]:
print('Achievements null counts:', cat_gen_df['Achievements'].isnull().sum())

cat_gen_df[['Name', 'Achievements']].iloc[8:13]

Achievements null counts: 0


,Name,Achievements
10,WARSAW,34
11,Cthulhu Realms,0
12,Clockwork Dungeon,25
13,Royal Battleships,19
14,Diary of Lucie,0


In [31]:
def process_achievements_and_descriptors(df):
    """Parse as total number of achievements."""
    df = df.copy()    
    return df

achiev_df = process_achievements_and_descriptors(cat_gen_df)
achiev_df['Achievements'].value_counts().head()

Achievements
0     27823
10     1658
12     1378
20     1239
15     1171
Name: count, dtype: int64

In [32]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process columns
    df = process_name_type(df)
    df = process_age(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)
    
    return df

partially_clean = process(raw_steam_data)
partially_clean.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,Header image,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Genres,Tags,Screenshots,Movies,Developer,Publisher
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,6,11,30,0,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,Perpetual FX Creative,Perpetual FX Creative
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,53,5,12,0,0,0,0,0,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,Rusty Moyher,Wild Rooster
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,True,0,3,0,0,0,0,0,0,0,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,Odd Critter Games,Odd Critter Games
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,False,0,50,8,17,0,0,0,0,0,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,Unusual Games,Unusual Games
5,1469160,Wartune Reborn,"Feb 26, 2021",50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,87,49,0,0,0,0,0,0,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...,7Road,7Road


### Processing Description Columns

In [33]:
partially_clean[['About the game']].isnull().sum()

About the game    76
dtype: int64

In [34]:
def export_data(df, filename):
    """Export dataframe to csv file, filename prepended with 'steam_'.
    
    filename : str without file extension
    """
    filepath = '../data/exports/steam_' + filename + '.csv'
    
    df.to_csv(filepath, index=False)
    
    print_name = filename.replace('_', ' ')
    print("Exported {} to '{}'".format(print_name, filepath))

In [35]:
def process_descriptions(df, export=False):
    """Export descriptions to external csv file then remove these columns."""

    # by default we don't export, useful if calling function later
    if export:
        # create dataframe of description columns
        description_data = df[['AppID', 'About the game']]
        
        export_data(description_data, filename='description_data')
    
    # drop description columns from main dataframe
    df = df.drop(['About the game'], axis=1)
    
    return df

desc_df = process_descriptions(partially_clean, export=True)

Exported description data to '../data/exports/steam_description_data.csv'


In [36]:
# inspect exported data
pd.read_csv('../data/exports/steam_description_data.csv').head()

,AppID,About the game
0,20200,Galactic Bowling is an exaggerated and stylize...
1,655370,THE LAW!! Looks to be a showdown atop a train....
2,1355720,HENOSIS™ is a mysterious 2D Platform Puzzler w...
3,1139950,ABOUT THE GAME Play as a hacker who has arrang...
4,1469160,Feel tired of auto-fight? Feel tired of boring...


### Processing Media Columns

In [37]:
image_cols = ['Header image', 'Screenshots']

for col in image_cols:
    print(col+':', desc_df[col].isnull().sum())

desc_df[image_cols].head()

Header image: 0
Screenshots: 35


,Header image,Screenshots
0,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...
1,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...
3,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...
4,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...
5,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...


In [38]:
no_screenshots = desc_df[desc_df['Screenshots'].isnull()]

print_steam_links(no_screenshots)

no_screenshots

Project Scav: https://store.steampowered.com/app/1601260
Nogibator: Way Of Legs: https://store.steampowered.com/app/754330
Born of Fire: https://store.steampowered.com/app/1081470
Convoy Mod Tools: https://store.steampowered.com/app/1231920
Willowbrooke Post: https://store.steampowered.com/app/938500
The Sands Whisper Your Name: https://store.steampowered.com/app/282960
Wwbit: https://store.steampowered.com/app/907680
CLOSED: https://store.steampowered.com/app/1131330
BoomTris: https://store.steampowered.com/app/787410
Primal Carnage: Extinction - Workshop Tool: https://store.steampowered.com/app/1366270
Falling Bullets: https://store.steampowered.com/app/1017560
The Haunting: Blood Water Curse (EARLY ACCESS): https://store.steampowered.com/app/697810
REVENGER: https://store.steampowered.com/app/1477120
A Land Fit For Heroes: https://store.steampowered.com/app/456210
God Test: https://store.steampowered.com/app/797660
Playground: https://store.steampowered.com/app/523080
For Inco: http

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Header image,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Genres,Tags,Screenshots,Movies,Developer,Publisher
956,1601260,Project Scav,"Sep 30, 2021",0 - 20000,0,0,14.99,0,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,0,1,0,0,0,0,0,0,Single-player,"Action,Adventure,Indie,Simulation,Early Access","Early Access,FPS,Post-apocalyptic,Shooter,Char...",NaN,NaN,"Cannabis,Meta","Smokehoe Studios,Dead Pixel Labs"
3466,754330,Nogibator: Way Of Legs,"Feb 14, 2018",0 - 20000,0,0,2.99,0,"['Russian', 'English']",https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,28,20,15,0,0,0,0,0,"Single-player,Steam Achievements","Casual,Indie","Indie,Casual",NaN,NaN,"TheDreik,Xitilon",Phoenix Reborn Games
7237,1081470,Born of Fire,"Sep 4, 2019",20000 - 50000,0,0,0.00,0,"['English', 'Simplified Chinese', 'Traditional...",https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,64,39,0,0,0,0,0,0,"Single-player,Multi-player,PvP,Online PvP","Free to Play,RPG,Strategy,Early Access","Roguelike Deckbuilder,Free to Play,Strategy,Ea...",NaN,NaN,Leaf Boat Studio,Leaf Boat Studio
13386,1231920,Convoy Mod Tools,"Jan 21, 2020",0 - 20000,0,0,0.00,0,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,1,0,0,0,0,0,0,0,Single-player,"Action,Adventure,Indie,Strategy","Strategy,Action,Adventure,Indie",NaN,NaN,Convoy Games,Indietopia Games
15314,938500,Willowbrooke Post,"Mar 27, 2019",0 - 20000,0,0,0.00,0,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,36,91,0,0,36,0,36,0,Single-player,"Indie,Simulation,Early Access","Simulation,Early Access,Indie,Management,Story...",NaN,NaN,Crunchy Sushi,Crunchy Sushi
17823,282960,The Sands Whisper Your Name,"Nov 27, 2014",20000 - 50000,0,0,0.00,0,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,True,True,0,6,0,0,0,0,0,0,0,Single-player,"Indie,Early Access","Indie,Early Access,Survival,Crafting,Exploration",NaN,NaN,NexyMedia,NexyMedia
19309,907680,Wwbit,"Aug 17, 2018",100000 - 200000,0,0,0.00,0,['Arabic'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,55,710,566,0,1149,8,0,10,0,Single-player,"Casual,Indie","Nudity,Sexual Content,Indie,Casual,FPS,Anime",NaN,NaN,1bit,1bit
20635,1131330,CLOSED,"Sep 3, 2019",0 - 20000,0,18,0.00,0,"['English', 'Japanese', 'Traditional Chinese']",https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,43,20,0,0,0,0,0,0,"Single-player,Partial Controller Support,Steam...","Action,Indie,RPG","Nudity,Action,Indie,Female Protagonist,RPG,Sin...",NaN,NaN,"Kazuya,tensun3d",赤いトンボ
25492,787410,BoomTris,"Oct 4, 2019",0 - 20000,0,0,0.00,0,['English'],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,0,1,0,0,0,0,0,0,Single-player,"Indie,Strategy,Early Access","Indie,Strategy,Early Access,Tower Defense",NaN,NaN,Vladimir Afonin,Vladimir Afonin
26712,1366270,Primal Carnage: Extinction - Workshop Tool,"Jul 17, 2020",50000 - 100000,0,0,0.00,0,[],https://cdn.akamai.steamstatic.com/steam/apps/...,True,False,False,0,1,0,0,0,0,0,0,0,Single-player,Action,Action,NaN,NaN,Circle 5 Studios,Circle 5 Studios


In [39]:
print('Movies null values:', desc_df['Movies'].isnull().sum())
print()

desc_df[desc_df['Movies'].notnull()]['Movies'].iloc[0]

Movies null values: 2380



'http://cdn.akamai.steamstatic.com/steam/apps/256863704/movie_max.mp4?t=1638854607'

In [40]:
def process_media(df, export=False):
    """Remove media columns from dataframe, optionally exporting them to csv first."""
    df = df[df['Screenshots'].notnull()].copy()
    
    if export:
        media_data = df[['AppID', 'Header image', 'Screenshots', 'Movies']]
        
        export_data(media_data, 'media_data')
        
    df = df.drop(['Header image', 'Screenshots', 'Movies'], axis=1)
    
    return df

media_df = process_media(desc_df, export=True)

Exported media data to '../data/exports/steam_media_data.csv'


In [41]:
# inspect exported data
pd.read_csv('../data/exports/steam_media_data.csv').head()

,AppID,Header image,Screenshots,Movies
0,20200,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1355720,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1139950,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1469160,https://cdn.akamai.steamstatic.com/steam/apps/...,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [42]:
print('Before removing data:\n')
achiev_df.info(verbose=False, memory_usage="deep")

print('\nData with descriptions and media removed:\n')
media_df.info(verbose=False, memory_usage="deep")

Before removing data:

<class 'pandas.core.frame.DataFrame'>
Index: 62711 entries, 0 to 85094
Columns: 30 entries, AppID to Publisher
dtypes: bool(3), category(1), float64(1), int64(12), object(13)
memory usage: 265.0 MB

Data with descriptions and media removed:

<class 'pandas.core.frame.DataFrame'>
Index: 62676 entries, 0 to 85094
Columns: 26 entries, AppID to Publisher
dtypes: bool(3), category(1), float64(1), int64(12), object(9)
memory usage: 56.9 MB


## Processing Release Date


In [43]:
def process_release_date(df):
    df = df.copy()

    df['Release date'] = pd.to_datetime(df['Release date'], format='%b %d, %Y', errors='coerce')
    
    df = df[df['Release date'].notnull()]
    
    return df

releasedate_df = process_release_date(media_df)
releasedate_df.head()


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Genres,Tags,Developer,Publisher
0,20200,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,['English'],True,False,False,0,6,11,30,0,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",Perpetual FX Creative,Perpetual FX Creative
1,655370,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,False,0,53,5,12,0,0,0,0,0,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",Rusty Moyher,Wild Rooster
3,1355720,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,True,0,3,0,0,0,0,0,0,0,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",Odd Critter Games,Odd Critter Games
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,"['English', 'Spanish - Spain']",True,True,False,0,50,8,17,0,0,0,0,0,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",Unusual Games,Unusual Games
5,1469160,Wartune Reborn,2021-02-26,50000 - 100000,68,0,0.00,0,['English'],True,False,False,0,87,49,0,0,0,0,0,0,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",7Road,7Road


In [44]:
def process(df):
    """Process data set. Will eventually contain calls to all functions we write."""
    
    # Copy the input dataframe to avoid accidentally modifying original data
    df = df.copy()
    
    # Remove duplicate rows - all appids should be unique
    df = df.drop_duplicates()
    
    # Remove collumns with more than 50% null values
    df = drop_null_cols(df)
    
    # Process columns
    df = process_name_type(df)
    df = process_age(df)
    df = process_developers_and_publishers(df)
    df = process_categories_and_genres(df)
    df = process_achievements_and_descriptors(df)  
    df = process_release_date(df)
    
    # Process columns which export data
    df = process_descriptions(df, export=True)
    df = process_media(df, export=True)
    
    return df

steam_data = process(raw_steam_data)
steam_data.head()

Exported description data to '../data/exports/steam_description_data.csv'
Exported media data to '../data/exports/steam_media_data.csv'


,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,Supported languages,Windows,Mac,Linux,User score,Positive,Negative,Achievements,Recommendations,Average playtime forever,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Categories,Genres,Tags,Developer,Publisher
0,20200,Galactic Bowling,2008-10-21,0 - 20000,0,0,19.99,0,['English'],True,False,False,0,6,11,30,0,0,0,0,0,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",Perpetual FX Creative,Perpetual FX Creative
1,655370,Train Bandit,2017-10-12,0 - 20000,0,0,0.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,False,0,53,5,12,0,0,0,0,0,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",Rusty Moyher,Wild Rooster
3,1355720,Henosis™,2020-07-23,0 - 20000,0,0,5.99,0,"['English', 'French', 'Italian', 'German', 'Sp...",True,True,True,0,3,0,0,0,0,0,0,0,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",Odd Critter Games,Odd Critter Games
4,1139950,Two Weeks in Painland,2020-02-03,0 - 20000,0,0,0.00,0,"['English', 'Spanish - Spain']",True,True,False,0,50,8,17,0,0,0,0,0,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",Unusual Games,Unusual Games
5,1469160,Wartune Reborn,2021-02-26,50000 - 100000,68,0,0.00,0,['English'],True,False,False,0,87,49,0,0,0,0,0,0,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip...","Turn-Based Combat,Massively Multiplayer,Multip...",7Road,7Road


## Final Steps

In [45]:
steam_data.isnull().sum()

AppID                         0
Name                          0
Release date                  0
Estimated owners              0
Peak CCU                      0
Required age                  0
Price                         0
DLC count                     0
Supported languages           0
Windows                       0
Mac                           0
Linux                         0
User score                    0
Positive                      0
Negative                      0
Achievements                  0
Recommendations               0
Average playtime forever      0
Average playtime two weeks    0
Median playtime forever       0
Median playtime two weeks     0
Categories                    0
Genres                        0
Tags                          0
Developer                     0
Publisher                     0
dtype: int64

In [46]:
raw_steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Columns: 39 entries, AppID to Movies
dtypes: bool(3), float64(2), int64(14), object(20)
memory usage: 378.5 MB


In [47]:
steam_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 62564 entries, 0 to 85094
Columns: 26 entries, AppID to Publisher
dtypes: bool(3), category(1), datetime64[ns](1), float64(1), int64(12), object(8)
memory usage: 53.2 MB


In [48]:
steam_data.to_csv('../data/exports/steam_partially_clean.csv', index=False)